In [1]:
import numpy as np
import scipy.linalg as la
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
import geomstats as gs
import geomstats.geometry.spd_matrices as spd
import pickle
from scipy.signal import butter, lfilter
from geomstats.learning.frechet_mean import FrechetMean
from scipy.io import loadmat
import pandas as pd
from copy import deepcopy
from scipy.stats import wilcoxon


pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.options.display.float_format = "{:,.3f}".format

INFO: Using numpy backend


# Preprocessing

In [2]:
def getCovariances(X):
    covlist = np.empty((X.shape[0], X.shape[2], X.shape[2]))
    for i in range(len(X)):
        trial = X[i]
        n = trial.shape[0]
        epoch = (trial - np.mean(trial))/np.std(trial)
        covlist[i] = np.dot(trial.T, trial)/n
    return covlist

def bandpass(X, fs, fl, fh, order):
    X_filtered = np.empty_like(X)    
    for i in range(X.shape[1]):
        trial = X[:, i, :]
        nyq = 0.5 * fs
        low = fl / nyq
        high = fh / nyq
        b, a = butter(order, [low, high], btype='band')
        trial = lfilter(b, a, trial, axis=0)
        X_filtered[:, i, :] = trial
            
    return X_filtered

def getData(session, fs=1000, fl=8, fh=30, order = 5):
    channels = [7, 8, 9, 10, 12, 13, 14, 17, 18, 19, 20, 32, 33, 34, 35, 36, 37, 38, 39, 40]
    SubjectsCOV = []
    SubjectsY = []
    for i in range(1, 55):
        filename = 'datasets/54subjects/'+ session + '/subj{}_EEG_MI.mat'.format(i)
        data = loadmat(filename)
        
        d_train = data['EEG_MI_train']
        Y_train = d_train['y_dec'][0][0][0] - 1
        
        d_test = data['EEG_MI_test']
        Y_test = d_test['y_dec'][0][0][0] - 1
        
        X_train = d_train['smt'][0][0][:, :, channels]
        X_train_filtered = bandpass(X_train, fs, fl, fh, order)
        X_train_filtered = X_train_filtered[1000:3500]
        
        X_test = d_test['smt'][0][0][:, :, channels]
        X_test_filtered = bandpass(X_test, fs, fl, fh, order)
        X_test_filtered = X_test_filtered[1000:3500]
        
        X = np.concatenate((X_train_filtered, X_test_filtered), axis = 1)
        X = np.swapaxes(X, 0, 1)
        covX = getCovariances(X)
        Y = np.concatenate((Y_train, Y_test))    
        SubjectsCOV.append(covX)
        SubjectsY.append(Y)
                    
    return SubjectsCOV, SubjectsY

In [4]:
# SubjectsCOV, SubjectsY = getData("sess01")
# filename = 'datasets/54COVSess01.pickle'
# outfile = open(filename,'wb')
# pickle.dump([SubjectsCOV, SubjectsY], outfile)
# outfile.close()

# SubjectsCOV, SubjectsY = getData("sess02")
# filename = 'datasets/54COVSess02.pickle'
# outfile = open(filename,'wb')
# pickle.dump([SubjectsCOV, SubjectsY], outfile)
# outfile.close()

# RCSP class

In [2]:
class RCSP:
    def __init__(self, metric, nchannels, clf):
        self.metric = metric
        self.nchannels = nchannels
        self.clf=clf()
        self.V=None
        self.n=None
    
    def estimateMeans(self, classSpecificCOV):
        if self.metric=="classic":
            class0_avg = sum(classSpecificCOV[0])/len(classSpecificCOV[0])
            class1_avg = sum(classSpecificCOV[1])/len(classSpecificCOV[1])
            return [class0_avg, class1_avg]
        elif self.metric=="AIRM":
            estimator = FrechetMean(spd.SPDMetricAffine(n=self.nchannels), max_iter=64)
        elif self.metric=="LEM":
            estimator = FrechetMean(spd.SPDMetricLogEuclidean(n=self.nchannels), max_iter=64)
        elif self.metric=="BW":
            estimator = FrechetMean(spd.SPDMetricBuresWasserstein(n=self.nchannels), max_iter=64) #doesn't work yet
        else:
            raise Exception("Not implemented metric")
            
        means = []
        
        for COV in classSpecificCOV:
            estimator.fit(COV)
            mean = estimator.estimate_
            means.append(mean)
        return means
    
    def separate_classes(self, X, Y):
        classSpecificCOV = []
        for i in range(2): 
            ind = np.where(Y==i)[0]
            classCOV = X[ind]
            classSpecificCOV.append(classCOV)
        return classSpecificCOV
    
    def CSP(self, means, n):
        _,V = la.eigh(means[0], means[0]+means[1])
        V = np.concatenate((V[:, :n], V[:, -n:]), axis=1)
        return V
    
    def applyCSP(self, trial, V):
        a = np.dot(np.dot(V.T, trial), V) 
        f = np.log(np.diagonal(a)/np.trace(a)) #logvariance features 
        return f
    
    def train(self, trainCOV, trainLabels, n=3):
        
        classSpecificCOV = self.separate_classes(trainCOV, trainLabels)
        means = self.estimateMeans(classSpecificCOV)
        
        V=self.CSP(means, n)
        self.V=V
        self.n=n
        train_features = np.empty((len(trainCOV), 2*n))
        
        for i in range(len(trainCOV)):
            trial = trainCOV[i]
            train_features[i] = self.applyCSP(trial, V)
        
        self.clf.fit(train_features, trainLabels)
    
    def predict(self, testCOV):
        V = self.V
        n = self.n
        
        if V is None or n is None:
            raise Exception('Train the model first')
        
        test_features = np.empty((len(testCOV), 2*n))
        for i in range(len(testCOV)):
            trial = testCOV[i]
            test_features[i] = self.applyCSP(trial, V)
        
        prediction = self.clf.predict(test_features)
        return prediction

# Globaly used things

In [3]:
#base classifiers 
c_csp = RCSP('classic', 20, LDA)
airm_csp = RCSP('AIRM', 20, LDA)
lem_csp = RCSP('LEM', 20, LDA)

In [4]:
def count_accuracy(predicted, true, dec_places=2):
    err_count = 0
    for j in range(len(true)):
        if predicted[j]!=true[j]:
            err_count+=1
    acc = (1-err_count/len(true))*100
    acc = round(acc, dec_places)
    return acc

In [5]:
SubjectsCOV1, SubjectsY1 = pickle.load(open('datasets/54COVSess01.pickle','rb'))
SubjectsCOV2, SubjectsY2 = pickle.load(open('datasets/54COVSess02.pickle','rb'))

#Values were extracted from this paper: https://www.researchgate.net/publication/330745291_EEG_dataset_and_OpenBMI_toolbox_for_three_BCI_paradigms_an_investigation_into_BCI_illiteracy

paper_s1 = [61,96,95,53,94,77,49,57,86,65,47,46,56,58,55,53,83,92,82,59,98,77,54,49,54,49,56,94,99,76,58,56,99,48,52,97,93,56,64,46,62,47,77,99,93,53,44,50,63,59,71,72,50,53,67.3]
paper_s2 = [83,86,94,57,81,88,71,66,71,61,59,58,54,48,57,69,42,82,89,73,100,85,68,54,57,44,70,97,98,66,57,97,89,47,52,94,81,52,52,58,48,63,86,100,99,58,59,49,62,58,52,72,54,45,68.6]

# Majority Vote

## Subject-Dependent

In [15]:
def majority_vote(SubjectsCOV, SubjectsY, n, papervals):
    df = pd.DataFrame(index=list(range(1, 55))+['Average', 'p-values'], columns=['Paper', 'Classic CSP', 'AIRM CSP', 'LEM CSP', 'Majority Vote'])
    
    c_results = [] #very sloppy coding, did not prioritize clean code
    a_results = []
    l_results = []
    m_results = []
    
    for i in range(len(SubjectsCOV)):
        COVi = SubjectsCOV[i]
        Yi = SubjectsY[i]

        trainCOV = COVi[:100]
        trainY = Yi[:100]

        testCOV = COVi[100:] 
        testY = Yi[100:] 

        c_csp.train(trainCOV, trainY, n=n)
        c_res = c_csp.predict(testCOV)
        airm_csp.train(trainCOV, trainY, n=n)
        a_res = airm_csp.predict(testCOV)

        lem_csp.train(trainCOV, trainY, n=n)
        l_res = lem_csp.predict(testCOV)

        res = np.column_stack((c_res, a_res, l_res))
        N = len(res)

        majority = np.empty(N, dtype='uint8')

        for j in range(N):
            majority[j] = np.argmax(np.bincount(res[j]))
    
        c_results.append(count_accuracy(c_res, testY))
        a_results.append(count_accuracy(a_res, testY))
        l_results.append(count_accuracy(l_res, testY))
        m_results.append(count_accuracy(majority, testY))
    
    c_results.append(sum(c_results)/len(c_results))
    _, pval = wilcoxon(papervals[:-1], c_results[:-1])
    c_results.append(pval)
    
    a_results.append(sum(a_results)/len(a_results))
    _, pval = wilcoxon(papervals[:-1], a_results[:-1])
    a_results.append(pval)
                     
    l_results.append(sum(l_results)/len(l_results))
    _, pval = wilcoxon(papervals[:-1], l_results[:-1])
    l_results.append(pval)
    
    m_results.append(sum(m_results)/len(m_results))
    _, pval = wilcoxon(papervals[:-1], m_results[:-1])
    m_results.append(pval)
    
    
    
    df['Paper'] = papervals + [None]
    df['Classic CSP'] = c_results
    df['AIRM CSP'] = a_results
    df['LEM CSP'] = l_results
    df['Majority Vote'] = m_results
    
    return df

### Session 1

#### N=3

In [16]:
df = majority_vote(SubjectsCOV1, SubjectsY1, 3, paper_s1)
df.head(56)

,Paper,Classic CSP,AIRM CSP,LEM CSP,Majority Vote
1,61.000,67.000,68.000,66.000,69.000
2,96.000,94.000,95.000,95.000,95.000
3,95.000,96.000,93.000,92.000,93.000
4,53.000,51.000,49.000,48.000,50.000
5,94.000,91.000,93.000,93.000,93.000
6,77.000,71.000,76.000,75.000,78.000
7,49.000,52.000,49.000,49.000,49.000
8,57.000,55.000,58.000,55.000,56.000
9,86.000,83.000,83.000,81.000,83.000
10,65.000,64.000,66.000,64.000,63.000


#### N = 4

In [17]:
df = majority_vote(SubjectsCOV1, SubjectsY1, 4, paper_s1)
df.head(56)

,Paper,Classic CSP,AIRM CSP,LEM CSP,Majority Vote
1,61.000,77.000,68.000,63.000,69.000
2,96.000,94.000,94.000,94.000,94.000
3,95.000,94.000,92.000,94.000,94.000
4,53.000,54.000,51.000,49.000,51.000
5,94.000,88.000,88.000,88.000,88.000
6,77.000,69.000,74.000,76.000,74.000
7,49.000,54.000,54.000,52.000,54.000
8,57.000,61.000,56.000,57.000,58.000
9,86.000,81.000,81.000,78.000,80.000
10,65.000,66.000,64.000,57.000,61.000


#### N = 5

In [18]:
df = majority_vote(SubjectsCOV1, SubjectsY1, 5, paper_s1)
df.head(56)

,Paper,Classic CSP,AIRM CSP,LEM CSP,Majority Vote
1,61.000,64.000,69.000,64.000,68.000
2,96.000,92.000,94.000,93.000,93.000
3,95.000,93.000,92.000,92.000,92.000
4,53.000,52.000,49.000,51.000,54.000
5,94.000,87.000,86.000,86.000,86.000
6,77.000,73.000,74.000,75.000,77.000
7,49.000,54.000,53.000,51.000,53.000
8,57.000,58.000,55.000,51.000,56.000
9,86.000,85.000,83.000,77.000,83.000
10,65.000,69.000,62.000,62.000,64.000


### Session 2

#### N=3

In [19]:
df = majority_vote(SubjectsCOV2, SubjectsY2, 3, paper_s2)
df.head(56)

,Paper,Classic CSP,AIRM CSP,LEM CSP,Majority Vote
1,83.000,80.000,78.000,77.000,79.000
2,86.000,91.000,95.000,94.000,95.000
3,94.000,94.000,94.000,95.000,94.000
4,57.000,62.000,60.000,61.000,61.000
5,81.000,81.000,77.000,81.000,80.000
6,88.000,86.000,86.000,83.000,83.000
7,71.000,70.000,63.000,61.000,63.000
8,66.000,58.000,69.000,66.000,66.000
9,71.000,65.000,73.000,71.000,71.000
10,61.000,62.000,67.000,68.000,67.000


#### N=4

In [20]:
df = majority_vote(SubjectsCOV2, SubjectsY2, 4, paper_s2)
df.head(56)

,Paper,Classic CSP,AIRM CSP,LEM CSP,Majority Vote
1,83.000,77.000,80.000,79.000,80.000
2,86.000,87.000,94.000,94.000,94.000
3,94.000,85.000,91.000,91.000,91.000
4,57.000,60.000,61.000,62.000,62.000
5,81.000,86.000,80.000,81.000,81.000
6,88.000,87.000,86.000,83.000,87.000
7,71.000,65.000,67.000,66.000,67.000
8,66.000,57.000,67.000,63.000,64.000
9,71.000,68.000,73.000,73.000,72.000
10,61.000,64.000,67.000,67.000,68.000


#### N=5

In [21]:
df = majority_vote(SubjectsCOV2, SubjectsY2, 5, paper_s2)
df.head(56)

,Paper,Classic CSP,AIRM CSP,LEM CSP,Majority Vote
1,83.000,78.000,81.000,75.000,79.000
2,86.000,89.000,93.000,94.000,94.000
3,94.000,79.000,87.000,92.000,87.000
4,57.000,61.000,65.000,63.000,66.000
5,81.000,85.000,82.000,81.000,83.000
6,88.000,87.000,85.000,82.000,85.000
7,71.000,68.000,66.000,70.000,67.000
8,66.000,61.000,63.000,64.000,62.000
9,71.000,69.000,73.000,74.000,72.000
10,61.000,62.000,65.000,69.000,68.000


## Subject-Independent

#### We don't have subject-independent baseline results from the paper, so Wilcoxon SR test compares my implementation of Classic CSP with everything else.

In [25]:
from copy import deepcopy

In [ ]:
def majority_vote_subjectindependent(SubjectsCOV, SubjectsY, n):
    df = pd.DataFrame(index=list(range(1, 55))+['Average', 'p-value'], columns=['Classic CSP', 'AIRM CSP', 'LEM CSP', 'Majority Vote'])
    
    c_results = [] #very sloppy coding, did not prioritize clean code
    a_results = []
    l_results = []
    m_results = []
    
    for i in range(len(SubjectsCOV)):
        SC = deepcopy(SubjectsCOV)
        SY = deepcopy(SubjectsY)
            
        testCOV = SC.pop(i)
        testY = SY.pop(i)
            
        trainCOV = None
        trainY = None
            
        for j in range(len(SubjectsCOV)-1):
            if trainCOV is None:
                trainCOV = SC[j]
                trainY = SY[j]
            else:
                trainCOV = np.concatenate((trainCOV, SC[j]))
                trainY = np.concatenate((trainY, SY[j]))

        c_csp.train(trainCOV, trainY, n=n)
        c_res = c_csp.predict(testCOV)

        airm_csp.train(trainCOV, trainY, n=n)
        a_res = airm_csp.predict(testCOV)

        lem_csp.train(trainCOV, trainY, n=n)
        l_res = lem_csp.predict(testCOV)

        res = np.column_stack((c_res, a_res, l_res))
        N = len(res)

        majority = np.empty(N, dtype='uint8')

        for j in range(N):
            majority[j] = np.argmax(np.bincount(res[j]))
    
        c_results.append(count_accuracy(c_res, testY))
        a_results.append(count_accuracy(a_res, testY))
        l_results.append(count_accuracy(l_res, testY))
        m_results.append(count_accuracy(majority, testY))
    
    c_results.append(sum(c_results)/len(c_results))

    a_results.append(sum(a_results)/len(a_results))
    _, pval = wilcoxon(a_results[:-1], c_results[:-1])
    a_results.append(pval)


    l_results.append(sum(l_results)/len(l_results))
    _, pval = wilcoxon(l_results[:-1], c_results[:-1])
    l_results.append(pval)

    m_results.append(sum(m_results)/len(m_results))
    _, pval = wilcoxon(m_results[:-1], c_results[:-1])
    m_results.append(pval)

    
    df['Classic CSP'] = c_results + [None]
    df['AIRM CSP'] = a_results
    df['LEM CSP'] = l_results
    df['Majority Vote'] = m_results
    
    return df

### Session 1

#### N = 3

In [8]:
df = majority_vote_subjectindependent(SubjectsCOV1, SubjectsY1, 3)
df.head(56)

ValueError: zero_method 'wilcox' and 'pratt' do not work if x - y is zero for all elements.

#### N = 4

In [ ]:
df = majority_vote_subjectindependent(SubjectsCOV1, SubjectsY1, 4)
df.head(56)

#### N = 5

In [ ]:
df = majority_vote_subjectindependent(SubjectsCOV1, SubjectsY1, 5)
df.head(56)

### Session 2

#### N = 3

In [ ]:
df = majority_vote_subjectindependent(SubjectsCOV2, SubjectsY2, 3)
df.head(56)

#### N =4 

In [ ]:
df = majority_vote_subjectindependent(SubjectsCOV2, SubjectsY2, 4)
df.head(56)

#### N =5 

In [ ]:
df = majority_vote_subjectindependent(SubjectsCOV2, SubjectsY2, 5)
df.head(56)

# Majority Vote with models with different hyperparameters

## Subject-dependent

In [ ]:
def majority_vote_hp(SubjectsCOV, SubjectsY, n_lowerlimit, n_higherlimit, papervals):
    df = pd.DataFrame(index=list(range(1, 55))+['Average', 'p-value'], columns=['Paper', 'Majority Vote 2'])
    
    m_results = []
    
    for i in range(len(SubjectsCOV)):
        COVi = SubjectsCOV[i]
        Yi = SubjectsY[i]

        trainCOV = COVi[:100]
        trainY = Yi[:100]

        testCOV = COVi[100:] 
        testY = Yi[100:] 
        
        res = np.empty((100, (n_higherlimit-n_lowerlimit+1)*3), dtype='uint8')
        
        c = 0
        for n in range(n_lowerlimit, n_higherlimit+1):
            c_csp.train(trainCOV, trainY, n=n)
            c_res = c_csp.predict(testCOV)
            res[:, c] = c_res
            c+=1
            
            airm_csp.train(trainCOV, trainY, n=n)
            a_res = airm_csp.predict(testCOV)
            res[:,c] = a_res
            c+=1
            
            lem_csp.train(trainCOV, trainY, n=n)
            l_res = lem_csp.predict(testCOV)
            res[:,c] = l_res
            c+=1
            
        N = len(res)

        majority = np.empty(N, dtype='uint8')

        for j in range(N):
            majority[j] = np.argmax(np.bincount(res[j]))
    
        m_results.append(count_accuracy(majority, testY))
    
    m_results.append(sum(m_results)/len(m_results))
    _, pval = wilcoxon(papervals[:-1], m_results[:-1])
    m_results.append(pval)

    
    df['Paper'] = papervals + [None]
    df['Majority Vote 2'] = m_results
    
    return df

### Session 1

In [ ]:
df = majority_vote_hp(SubjectsCOV1, SubjectsY1, 3, 5, paper_s1)

In [ ]:
df.head(56)

### Session 2

In [ ]:
df = majority_vote_hp(SubjectsCOV2, SubjectsY2, 3, 5, paper_s2)

In [ ]:
df.head(56)

# Bootstrap Mean Estimation

In [41]:
class RCSP_bootstrap(RCSP):
    def train(self, trainCOV, trainLabels, n=3, btsp_size=0.6, btsp_n=10):
        btsp_means = np.zeros((2, btsp_n, self.nchannels, self.nchannels))
        trials = len(trainCOV)
        idxs = list(range(trials))
        
        for i in range(btsp_n):
            idx = np.random.choice(idxs, int(btsp_size*trials))
            subsetCOV = trainCOV[idx]
            subsetY = trainLabels[idx]
            
            classSpecificCOV = self.separate_classes(subsetCOV, subsetY)
            means = self.estimateMeans(classSpecificCOV)
            btsp_means[0, i, :, :] = means[0]
            btsp_means[1, i, :, :] = means[1]
        
        btsp_means = self.estimateMeans(btsp_means)
        
        V = self.CSP(btsp_means, n)
        self.V=V
        self.n=n
        
        train_features = np.empty((len(trainCOV), 2*n))
        
        for i in range(len(trainCOV)):
            trial = trainCOV[i]
            train_features[i] = self.applyCSP(trial, V)
        
        self.clf.fit(train_features, trainLabels)    

In [42]:
c_rcsp_b = RCSP_bootstrap("classic", 20, LDA)
a_rcsp_b = RCSP_bootstrap("AIRM", 20, LDA)
l_rcsp_b = RCSP_bootstrap("LEM", 20, LDA)

In [56]:
def bootstrap_estimation(SubjectsCOV, SubjectsY, n, btsp_size, btsp_n, papervals, subject_independent=False):
    df = pd.DataFrame(index=list(range(1, 55))+['Average', 'p-values'], columns=['Paper', 'Bootstraped Classic CSP', 'Bootstraped AIRM CSP', 'Bootstraped LEM CSP'])
    
    c_results = []
    a_results = []
    l_results = []
    
    for i in range(len(SubjectsCOV)):
        if subject_independent:
            
            SC = deepcopy(SubjectsCOV)
            SY = deepcopy(SubjectsY)    
            testCOV = SC.pop(i)
            testY = SY.pop(i)
            
            trainCOV = None
            trainY = None
            
            for j in range(len(SubjectsCOV)-1):
                if trainCOV is None:
                    trainCOV = SC[j]
                    trainY = SY[j]
                else:
                    trainCOV = np.concatenate((trainCOV, SC[j]))
                    trainY = np.concatenate((trainY, SY[j]))
        else:
            COVi = SubjectsCOV[i]
            Yi = SubjectsY[i]

            trainCOV = COVi[:100]
            trainY = Yi[:100]

            testCOV = COVi[100:] 
            testY = Yi[100:] 
        
        c_rcsp_b.train(trainCOV, trainY, n, btsp_size, btsp_n)
        c_res = c_rcsp_b.predict(testCOV)
        
        a_rcsp_b.train(trainCOV, trainY, n, btsp_size, btsp_n)
        a_res = a_rcsp_b.predict(testCOV)
        
        l_rcsp_b.train(trainCOV, trainY, n, btsp_size, btsp_n)
        l_res = l_rcsp_b.predict(testCOV)
        
        c_results.append(count_accuracy(c_res, testY))
        a_results.append(count_accuracy(a_res, testY))
        l_results.append(count_accuracy(l_res, testY))
        
    c_results.append(sum(c_results)/len(c_results))
    a_results.append(sum(a_results)/len(a_results))
    l_results.append(sum(l_results)/len(l_results)) 
    
    if not subject_independent:
        c_results.append(wilcoxon(papervals[:-1], c_results[:-1])[1])
        a_results.append(wilcoxon(papervals[:-1], a_results[:-1])[1])
        l_results.append(wilcoxon(papervals[:-1], l_results[:-1])[1])
    
    df['Paper'] = papervals + [None]
    df['Bootstraped Classic CSP'] = c_results
    df['Bootstraped AIRM CSP'] = a_results
    df['Bootstraped LEM CSP'] = l_results
    
    if subject_independent:
        df = df.drop(['Paper'], axis=1)
        df = df.drop(['p-values'], axis = 0)
    return df
        

## Subject-Dependent

### Session 1

In [57]:
df = bootstrap_estimation(SubjectsCOV1, SubjectsY1, n=4, btsp_size=0.8, btsp_n=10, papervals = paper_s1, subject_independent=False)
df.head(56)

ValueError: The samples x and y must have the same length.

### Session 2

In [ ]:
df = bootstrap_estimation(SubjectsCOV2, SubjectsY2, n=4, btsp_size=0.8, btsp_n=10, papervals = paper_s2, subject_independent=False)
df.head(56)

## Subject Independent

### Session 1

In [ ]:
df = bootstrap_estimation(SubjectsCOV1, SubjectsY1, n=4, btsp_size=0.8, btsp_n=10, papervals = paper_s1, subject_independent=True)
df.head(56)

### Session 2

In [ ]:
df = bootstrap_estimation(SubjectsCOV2, SubjectsY2, n=4, btsp_size=0.8, btsp_n=10, papervals = paper_s2, subject_independent=True)
df.head(56)

# Bootstrap Aggregation

In [ ]:
def bootstrap_aggregation():
    

# Boosting